In [2]:
%load_ext twisted_ipython
%autoawait twisted

The twisted_ipython extension is already loaded. To reload it, use:
  %reload_ext twisted_ipython


In [133]:
from twisted.internet import reactor
import attr
from txdbus import client


class _Key:
    pass

  
DBUS_TYPE = _Key()
DBUS_TYPE_PARAMS = _Key()


@attr.s
class Notification:
    appname = attr.ib(default='', metadata={DBUS_TYPE:'s'})
    replaces = attr.ib(default=0, metadata={DBUS_TYPE: 'u'})
    icon = attr.ib(default='', metadata={DBUS_TYPE: 's'})
    summary = attr.ib(default='', metadata={DBUS_TYPE: 's'})
    message = attr.ib(default='', metadata={DBUS_TYPE: 's'})
    actions = attr.ib(default=[], metadata={DBUS_TYPE: 'as'})
    hints = attr.ib(default={}, metadata={DBUS_TYPE: 'ae', DBUS_TYPE_PARAMS: ('s', 'v')})
    timeout = attr.ib(default=1000, metadata={DBUS_TYPE: 'i'})

In [134]:
STRUCTURE_TYPES = {
    'r': ('(', ')'),
    'e': ('{', '}')
}

def dbus_signature(typ):
    """
    class -> dbus signature
    """
    if isinstance(typ, list) or isinstance(typ, tuple):
        return ''.join([
            dbus_signature(subtype)
            for subtype in typ
        ])

    signature = ''
    
    for attr in typ.__attrs_attrs__:
        if DBUS_TYPE_PARAMS in attr.metadata:
            base_type = attr.metadata[DBUS_TYPE]
            params = attr.metadata[DBUS_TYPE_PARAMS]
            
            for t in base_type:
                if t in STRUCTURE_TYPES:
                    left, right = STRUCTURE_TYPES[t]
                    signature += left
                    # May either be a list of elements
                    # or a single set
                    if type(params) == list:
                        param = params.pop(0)
                    else:
                        param = params
                        
                    # May be a single string or a tuple
                    if type(param) == str:
                       param = [param]
                    for p in param:
                        if type(p) == str:
                            signature += p
                        else:
                            signature += dbus_signature(p)
                    signature += right
                else:
                   signature += t
        else:
            signature += attr.metadata[DBUS_TYPE]
        
    return signature

In [135]:
dbus_signature(Notification)

'susssasa{sv}i'

In [163]:
def dbus_marshall(data_structure):
    """
    data structure -> thing txdbus can send
    """
    if isinstance(data_structure, list) or isinstance(data_structure, tuple):
        rv = [
            dbus_marshall(element)
            for element in data_structure
        ]
        return [rv]
    elif isinstance(data_structure, dict):
        return [{k: dbus_marshall(v)[0] for k, v in data_structure.items()}]
    elif hasattr(data_structure.__class__, '__attrs_attrs__'):
        ret = []
        attrs = data_structure.__class__.__attrs_attrs__

        for attr in attrs:
            value = getattr(data_structure, attr.name)
            ret += dbus_marshall(value)
        return ret
    else:
       return [data_structure]

In [164]:
dbus_marshall(Notification(appname='jupyter', summary='an important message', message='my dog', timeout=5000, hints={'key': 'bro'}))

['jupyter', 0, '', 'an important message', 'my dog', [], {'key': 'bro'}, 5000]

In [165]:
conn = await client.connect(reactor, 'session')

notifier = await conn.getRemoteObject('org.freedesktop.Notifications', '/org/freedesktop/Notifications')

In [171]:
from txdbus.interface import DBusInterface, Method

iface = DBusInterface(
    'org.freedesktop.Notifications',
    Method('Notify', arguments=dbus_signature(Notification), returns='u')
)

unreflected = await conn.getRemoteObject(
    'org.freedesktop.Notifications',
    '/org/freedesktop/Notifications',
    iface
)

await unreflected.callRemote('Notify', *dbus_marshall(Notification(appname='jupyter', summary='an important message', message='my dog', timeout=5000)))

47

In [174]:
def dbus_unmarshall(raw, typ):
    # TODO
    pass